# Data Cleaner

Different scripts to clean and trim our data

In [7]:
from db_scripts import *

GWAS dataset:

In [14]:
fileIn = getPath('gwas_catalog_v1.0-associations_e108_r2023-01-14.tsv') # https://www.ebi.ac.uk/gwas/docs/file-downloads
fileOut = getPath('gwas_trimmed.tsv')

data = pd.read_csv(fileIn, sep='\t', low_memory=False)    # Reads gwas tsv
data = data.loc[data['DISEASE/TRAIT']=='Type 1 diabetes']   # Select only rows regarding type 1 diabetes
data = data.loc[data['CHR_ID']=='6']                        # Select only rows for chromosome 6
data = data[["SNPS","REGION","CHR_POS","P-VALUE","MAPPED_GENE"]] # maybe also include STRONGEST SNP-RISK ALLELE and RISK ALLELE FREQUENCY ?

renameDict={}
for col in data.columns:
    newCol = re.sub(r'\W+', '_', col)   # Replaces special characters and whitespace with underscores
    renameDict.update({col:newCol})
data=data.rename(columns=renameDict)

data=removeDupeSNP(data)      # Remove duplicates (leaving the entry with largest p value)

if os.path.exists(fileOut):        # If the file exists,
    os.remove(filepath)             # delete it.
data.to_csv(filepath, sep='\t')

Functional data:

In [20]:
fileIn=getPath('Func_data.tsv')
fileOut=getPath('Func_data_clean.tsv')

data=pd.read_csv(fileIn,sep='\t')

# print(data) # multiple duplicate rs values, need to decide way to break ties
dupeList = data.duplicated(subset=['#Uploaded_variation','Allele'])   # Get list of duplicate values
filtered=data[~dupeList]
filtered.to_csv(fileOut, sep='\t')